### Notebook to be used to test that ATCamera, ATHeaderService and ATArchiver are online and working

In [ ]:
import numpy as np
from lsst.ts import salobj
import wget
import asyncio
from astropy.io import fits
from astropy import time as astropytime

import warnings
#import matplotlib.pyplot as plt  # imported as py above
from astropy.modeling import models, fitting
from scipy.ndimage.filters import gaussian_filter
from matplotlib import pyplot as plt
#%matplotlib ipympl
plt.rcParams['figure.figsize'] = [7, 6]

import lsst.daf.persistence as dafPersist
import matplotlib.pyplot as plt
%matplotlib inline
import lsst.afw.display as afwDisplay

afwDisplay.setDefaultBackend('matplotlib')
import time
import lsst.afw.cameraGeom.utils as cameraGeomUtils
import lsst.geom

from scipy.signal import medfilt
import copy
from scipy.ndimage.filters import gaussian_filter
from scipy import ndimage

import os
import logging
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
logger = logging.getLogger('image_display_notebook')

In [ ]:
#Import CWFS package
from lsst import cwfs
from lsst.cwfs.instrument import Instrument
from lsst.cwfs.algorithm import Algorithm
from lsst.cwfs.image import Image, readFile, aperture2image, showProjection
import lsst.cwfs.plots as plots

In [ ]:
# Want to display in firefly?
afwDisplay.setDefaultBackend('firefly')
os.environ['FIREFLY_HTML'] = "slate.html"
os.environ['FIREFLY_URL'] = 'http://139.229.170.210:8080/firefly/'

In [ ]:
# # Setup the butler
#accs_images = True
accs_images = False
if accs_images:
    repo = os.path.join("/home/saluser/ingest/accs/")#, mapper={'calibRoot': "/home/saluser/ingest/dmcs/CALIB"})
    butler = dafPersist.Butler(repo)
else:
    #repo = os.path.join("/home/saluser/ingest/dmcs/")#, mapper={'calibRoot': "/home/saluser/ingest/dmcs/CALIB"})
    repo = os.path.join("/mnt/dmcs/oods_butler_repo/repo/")
    butler = dafPersist.Butler(repo) #
    #butler = dafPersist.Butler(repo, mapper="lsst.obs.lsst.auxTel.AuxTelMapper")
#test   
#raw = butler.get("raw", visit=2019111300004)
#image = raw.getImage().array

In [ ]:
# Load the LATISS class
from lsst.ts.standardscripts.auxtel.latiss import LATISS

latiss = LATISS()

await latiss.start_task 

In [ ]:
# Load the ATTCS class
from lsst.ts.standardscripts.auxtel.attcs import ATTCS

attcs = ATTCS(domain=latiss.domain)

await attcs.start_task 

## Perform State Transitions

In [ ]:
# Get state of CSC
salobj.State((attcs.athexapod.evt_summaryState.get()).summaryState)

In [ ]:
# State transitions
#await salobj.set_summary_state(latiss.atspec, salobj.State.OFFLINE, timeout=60, settingsToApply='current')

In [ ]:
#Setup LATISS
#await latiss.atspec.cmd_homeLinearStage.start(timeout=90)
#await latiss.setup_atspec(filter='blank_bk7_wg05', grating='empty_1', linear_stage=60)

In [ ]:
# Declare target name
target_name='HD32309'

In [ ]:
logger = salobj.Controller("Script", index=1)
logger.evt_logMessage.set_put(message="Test Starting for target: {}".format(target_name))

## Declare focus offsets

In [ ]:
dz = 0.8
offset_intra = {'m1': 0.0,
          'm2': 0.0,
          'x': 0.0,
          'y': 0.0,
          'z': -dz,
          'u': 0.0,
          'v': 0.0
          }
offset_extra = {'m1': 0.0,
          'm2': 0.0,
          'x': 0.0,
          'y': 0.0,
          'z': 2*dz,
          'u': 0.0,
          'v': 0.0
          }

## Declare grating, filters, expTime, group_id, image_type

In [ ]:
filter='blank_bk7_wg05'
grating='ronchi90lpmm'

## Move to intra-focal image (-z on hexapod)

In [ ]:
# Move intra-focus
attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_offset.set_start(**offset_intra)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)

### Take an exposure

In [ ]:
expTime=45.0 # seconds
group_id=astropytime.Time.now().tai.isot

intra_endReadout = await latiss.take_image(exptime=expTime, shutter=True,image_type='ENGTEST',
                                           group_id=group_id, filter=filter, grating=grating) # 'blank_bk7_wg05' KPNO_406_828nm'  'KPNO_1111_436nm' 'KPNO_373A_677nm' 'ronchi170lpmm'


In [ ]:
# OODS event not in currently deployed XML (4.4.1)
#test= await ATArchiver.evt_imageInOODS.next(flush=False, timeout=30) 

In [ ]:
logger.evt_logMessage.set_put(message="CCS intraImage for target: {}".format(intra_endReadout.imageName))

## Move to extra-focal image  (+z on hexapod)

In [ ]:
# Move extra-focus
attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_offset.set_start(**offset_extra)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)

### Take an exposure

In [ ]:
#First flush events that we want to listen to
extra_endReadout = await latiss.take_image(exptime=expTime, shutter=True,image_type='ENGTEST',
                                           group_id=group_id, filter=filter, grating=grating)# 'blank_bk7_wg05' KPNO_406_828nm'  'KPNO_1111_436nm' 'KPNO_373A_677nm' 'ronchi170lpmm'


In [ ]:
# OODS event not in currently deployed XML (4.4.1)
#test= await ATArchiver.evt_imageInOODS.next(flush=False, timeout=30) 

In [ ]:
logger.evt_logMessage.set_put(message="CCS extraImage for target: {}".format(extra_endReadout.imageName))

## Move Hexapod back to focus Position

In [ ]:
#Move the hexapod back
# move back to zero offset
attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_offset.set_start(**offset_intra)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)

In [ ]:
#First flush events that we want to listen to
focus_endReadout = await latiss.take_image(exptime=expTime/6, shutter=True,image_type='ENGTEST',
                                           group_id=group_id, filter=filter, grating=grating)# 'blank_bk7_wg05' KPNO_406_828nm'  'KPNO_1111_436nm' 'KPNO_373A_677nm' 'ronchi170lpmm'


In [ ]:
# parse out visitID from filename - this is highly annoying
tmp=focus_endReadout.imageName.split('_')
prefix=tmp[2] # dayobs without the dashes

# Don't remember why I used int here... whitespace? 
# surely fixable but bigger fish.
suffix='{:05d}'.format(int(tmp[3].split('-')[0])) # SEQNUM, but need to trim extra 0 in obsid
focus_visitID = int((prefix+suffix))
logger.evt_logMessage.set_put(message="focusImage visitID for target: {}".format(focus_visitID))
print("focusImage visitID for target: {}".format(focus_visitID))

## Reduce Intra Data

In [ ]:
# parse out visitID from filename - this is highly annoying
tmp=intra_endReadout.imageName.split('_')
prefix=tmp[2] # dayobs without the dashes

# Don't remember why I used int here... whitespace? 
# surely fixable but bigger fish.
suffix='{:05d}'.format(int(tmp[3].split('-')[0])) # SEQNUM, but need to trim extra 0 in obsid
intra_visitID = int((prefix+suffix))
logger.evt_logMessage.set_put(message="intraImage visitID for target: {}".format(intra_visitID))

In [ ]:
## Load file from Butler
import importlib
import utils.processExposure 
importlib.reload(utils.processExposure)
from utils.processExposure import processExposure

In [ ]:
if False:
    print('MANUAL OVERRIDE ON intra_visitID')
    intra_visitID = 2020012700579
    print(intra_visitID)

# Grab image from butler, but have to redefine butler after each image
butler = dafPersist.Butler(repo)
exposure = butler.get("raw", visit=intra_visitID)
#image = raw.getImage().array

# do ISR correction
intra_isr_corr_exposure = processExposure(exposure, repo=repo, bias=None, flat=False, defects=False )
#intra_isr_corr_exposure = exposure

In [ ]:
#Cosmic Ray Repair
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask
imCharConfig = CharacterizeImageTask.ConfigClass()
imCharConfig.doMeasurePsf = False
imCharConfig.doApCorr = False
imCharConfig.doDeblend = False
imCharTask = CharacterizeImageTask(config=imCharConfig)
intra_crr_isr_corr_exposure = imCharTask.run(intra_isr_corr_exposure).exposure

In [ ]:
if False: # display the image in firefly
    plt.close('all')
    disp = afwDisplay.Display(2, reopenPlot=True)
    disp.setMaskPlaneColor('SAT', afwDisplay.IGNORE)
    disp.setImageColormap('gray')
    disp.scale('linear', 'zscale')
    disp.mtv(intra_crr_isr_corr_exposure, title='visit = {}'.format(intra_visitID))
    #cgUtils.overlayCcdBoxes(isr_corr_exposure.getDetector(), isTrimmed=True, display=disp)

## Reduce Extra Data

In [ ]:
# parse out visitID from filename - this is highly annoying
tmp=extra_endReadout.imageName.split('_')
prefix=tmp[2] # dayobs without the dashes

# Don't remember why I used int here... whitespace? 
# surely fixable but bigger fish.
suffix='{:05d}'.format(int(tmp[3].split('-')[0])) # SEQNUM, but need to trim extra 0 in obsid
extra_visitID = int((prefix+suffix))

In [ ]:
## Load file from Butler
import importlib
import utils.processExposure 
importlib.reload(utils.processExposure)
from utils.processExposure import processExposure

In [ ]:
if False:
    print('MANUAL OVERRIDE ON extra_visitID')
    extra_visitID = 2020012700580
    print(extra_visitID)

# Grab image from butler, but have to redefine butler after each image
butler = dafPersist.Butler(repo)
exposure = butler.get("raw", visit=extra_visitID)
#image = raw.getImage().array

# do ISR correction
extra_isr_corr_exposure = processExposure(exposure, repo=repo, bias=None, flat=False, defects=False )
#extra_isr_corr_exposure = exposure

In [ ]:
#Cosmic Ray Repair
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask
imCharConfig = CharacterizeImageTask.ConfigClass()
imCharConfig.doMeasurePsf = False
imCharConfig.doApCorr = False
imCharConfig.doDeblend = False
imCharTask = CharacterizeImageTask(config=imCharConfig)
extra_crr_isr_corr_exposure = imCharTask.run(extra_isr_corr_exposure).exposure

In [ ]:
if False: # display the image in firefly
    plt.close('all')
    disp = afwDisplay.Display(2, reopenPlot=True)
    disp.setMaskPlaneColor('SAT', afwDisplay.IGNORE)
    disp.setImageColormap('gray')
    disp.scale('linear', 'zscale')
    disp.mtv(extra_isr_corr_exposure, title='visit = {}'.format(extra_visitID))
    #cgUtils.overlayCcdBoxes(isr_corr_exposure.getDetector(), isTrimmed=True, display=disp)

In [ ]:
extra_visitID

In [ ]:
# # Look at the summed images
# coadd=copy.deepcopy(extra_isr_corr_exposure)
# coadd.image+=intra_isr_corr_exposure.image

# if False: # display the image in firefly
#     plt.close('all')
#     disp = afwDisplay.Display(2, reopenPlot=True)
#     disp.setMaskPlaneColor('SAT', afwDisplay.IGNORE)
#     disp.setImageColormap('gray')
#     disp.scale('linear', 'zscale')
#     disp.mtv(coadd, title='visit = {}'.format(extra_visitID))
#     #cgUtils.overlayCcdBoxes(isr_corr_exposure.getDetector(), isTrimmed=True, display=disp)

In [ ]:
# side=400
# intra_center = lsst.geom.Point2D(2039,1967)
# bbox_intra= lsst.geom.Box2I.makeCenteredBox(intra_center, lsst.geom.Extent2I(side, side))


In [ ]:
intra_exp = intra_crr_isr_corr_exposure.image.array
extra_exp = extra_isr_corr_exposure.image.array

### Grab centroid of images

In [ ]:
# Define the image it will use to perform final center
im_shape=intra_exp.shape
side=400
# Set to False to declare centroid manually!
if True:
    im= (intra_exp+extra_exp)
    im_filtered = medfilt(im,[5,5])
    im_filtered -= int(np.median(im_filtered))
    mean = np.mean(im_filtered)
#     im_filtered[im_filtered < mean] = 0.
#     im_filtered[im_filtered > mean] = 1.
    # iter 1
    ceny, cenx = np.array(ndimage.measurements.center_of_mass(im_filtered), dtype=int)
    # iter 2
    intra_square = intra_exp[ceny-side:ceny+side, cenx-side:cenx+side] 
    extra_square = extra_exp[ceny-side:ceny+side, cenx-side:cenx+side]
    im= (intra_square+extra_square)
    im_filtered = medfilt(im,[5,5])
    im_filtered -= int(np.median(im_filtered))
    mean = np.mean(im_filtered)
    im_filtered[im_filtered < mean] = 0.
    im_filtered[im_filtered > mean] = 1.
    # iter 1
    cy2, cx2 = np.array(ndimage.measurements.center_of_mass(im_filtered), dtype=int)
    print(cy2-side, cx2-side)    
    ceny += (cy2-side)
    cenx += (cx2-side)
else:
    # Manually declare where the center is
    cenx=2033 #round(im_shape[0]/2)
    ceny=1952 # round(im_shape[1]/2)
    
side=140 # side length of image
print('Creating stamps of centroid [y,x] = [{},{}] with a side length of {} pixels'.format(ceny,cenx,side))
im_shape=intra_exp.shape
intra_square = intra_exp[ceny-side:ceny+side, cenx-side:cenx+side] 
extra_square = extra_exp[ceny-side:ceny+side, cenx-side:cenx+side]

In [ ]:
#plt.imshow(extra_square)

In [ ]:
#Create object for CWFS code
# Select where your object is, but ours will be on-axis
fieldXY = [0.0,0.0]

# I1/I2 get modified down below, so reset here
I1 = None; I2=None
I1 = Image(intra_square, fieldXY, Image.INTRA)
I2 = Image(extra_square, fieldXY, Image.EXTRA)

## Load instrument profile from "/home/saluser/develop/cwfs/data/auxtel"
### Make sure pixelSize and offset are correct for the dataset! 
### You *must* modify the file and not the loaded object!!!

In [ ]:
# Declare instrument
inst=Instrument('auxtel_LATISS',I1.sizeinPix) # example
hex_to_focus_scale = 41.0
offset=0.5 *hex_to_focus_scale # [mm] multiply hexapod dz by magnification factor
pixelsize = 10e-6 # no binning for LATISS
# FIXME: put an assertion here and calculate binning above based on change in image size, also pull offset from filename!

print('Offset should be :{} [mm] at the focus, {} [mm] at the hexapod'.format(offset, offset/hex_to_focus_scale))
print('Offset in file is :{} [mm] at the focus'.format(1e3*inst.offset))
print('pixelSize should be: {}'.format(pixelsize))

#declare algorithm
# declare algorithm - exponential solver.
algo=Algorithm('exp',inst,1) # example - but only want 11 Zernikes (Num_of_Zernikes)

In [ ]:
#algo.numTerms = 11
#algo.ZTerms = np.arange(algo.numTerms) + 1

In [ ]:
# Plot images
plots.plotImage(I1.image,'intra') 
#plots.plotImage(I_focus.image,'focus')
plots.plotImage(I2.image,'extra')
print('Intra-focal visitID: {}'.format(intra_visitID))
print('Extra-focal visitID: {}'.format(extra_visitID))
print('In-focus visitID: {}'.format(focus_visitID))

In [ ]:
# Calculate zernikes and wavefront
# Note that this will change I1 and I2!
# if you ever want to rerun this with different parameters you have to reload I1 and I2!
start_time=time.time()
algo.runIt(inst,I1,I2,'onAxis')
end_time=time.time()
print('time to run fitting is {} [s]'.format(end_time-start_time))

In [ ]:
#print zernikes
print(algo.zer4UpNm)

In [ ]:
# plot the zernikes
plots.plotZer(algo.zer4UpNm[0:9],'nm')

In [ ]:
fig1 = plt.figure(1, figsize=(12,8))
ax11 = fig1.add_subplot(121)
ax11.set_title("defocus 0.8 - intra")
ax11.imshow(I1.image0)
ax11.contour(algo.pMask) 
ax12 = fig1.add_subplot(122)
ax12.set_title("defocus 0.8 - extra")
ax12.imshow(I2.image0)
ax12.contour(algo.pMask) 

In [ ]:
# Plot images with masks to check mapping is correct
# I think these plot residuals, but I1.image0 plots the original image
plots.plotImage(I1.image,'intra')#, mask=algo.pMask) 
#plots.plotImage(I_focus.image,'focus')
plots.plotImage(I2.image,'extra')#, mask=algo.pMask)

In [ ]:
plots.plotImage(algo.Wconverge,'Final wavefront')

In [ ]:
#plots.plotImage(algo.Wconverge,'Final wavefront with pupil mask applied', mask=algo.pMask)

In [ ]:
# Look at estimated wavefront and residual of wavefront that is not well fit
nanMask = np.ones(I1.image.shape)
nanMask[I1.pMask==0] = np.nan
fig, ax = plt.subplots(1,2, figsize=[10,4])
img = ax[0].imshow(algo.Wconverge*nanMask, origin='lower')
ax[0].set_title('Final WF = estimated + residual')
fig.colorbar(img, ax=ax[0])
img = ax[1].imshow(algo.West*nanMask, origin='lower')
ax[1].set_title('residual wavefront')
fig.colorbar(img, ax=ax[1])

In [ ]:
# Plot the residuals at each image - ideally they would be a constant
fig, ax = plt.subplots(1,2, figsize=[10,4])
img = ax[0].imshow(I1.image, origin='lower')
ax[0].set_title('Intra residual image')
fig.colorbar(img, ax=ax[0])
img = ax[1].imshow(I2.image, origin='lower')
ax[1].set_title('Extra residual image')
fig.colorbar(img, ax=ax[1])

In [ ]:
# Only here for me to debug, should be up top with other declarations
import importlib
import utils.findNarrowbandRonchiPeaks
importlib.reload(utils.findNarrowbandRonchiPeaks)
from utils.findNarrowbandRonchiPeaks import findNarrowbandRonchiPeaks

import utils.fitExposure
importlib.reload(utils.fitExposure)
from utils.fitExposure import fit2DGaussian

import utils.calc_CofM
importlib.reload(utils.calc_CofM)
from utils.calc_CofM import calc_CofM

import utils.calc_encircled_energy
importlib.reload(utils.calc_encircled_energy)
from utils.calc_encircled_energy import calc_encircled_energy

In [ ]:
# Source detection libraries
from lsst.meas.algorithms.detection import SourceDetectionTask
import lsst.afw.table as afwTable

# create the output table for source detection
schema = afwTable.SourceTable.makeMinimalSchema()
config = SourceDetectionTask.ConfigClass()
config.thresholdValue = 10  # detection threshold after smoothing
sourceDetectionTask = SourceDetectionTask(schema=schema, config=config)

## Fit a gaussian

In [ ]:
# Declare approximation of where the zero-order star is
zeroth_order_estimate = lsst.geom.Point2D(1700,1960)
zeroth_order_estimate = lsst.geom.Point2D(1650,1930)
zeroth_order_estimate = lsst.geom.Point2D(2100,2100)

fit_data=[]

for index, img_name in enumerate(image_list):
    # parse out visitID from filename - this is highly annoying
    print('Processing file {} of {}, filename={}'.format(index,len(image_list), img_name))
    tmp=img_name.split('_')
    prefix=tmp[2] # dayobs without the dashes
    # Don't remember why I used int here... whitespace? 
    # surely fixable but bigger fish.
    suffix='{:05d}'.format(int(tmp[3].split('-')[0])) # SEQNUM, but need to trim extra 0 in obsid
    visitID = int((prefix+suffix))
    dataId1 = {'visit': visitID}
    #multi_file_dataset[i]['visitID']=visitID
    
    #exposure = butler.get('raw', **dataId1)
    exposure = await grabATImage(visitID, repo, timeout = 40, poll_freq_hz=2)
    # do ISR correction
    isr_corr_exposure = processExposure(exposure, repo=repo, bias=None, defects=None)
    
    if False: # display the image in firefly
        plt.close('all')
        disp = afwDisplay.Display(2, reopenPlot=True)
        disp.setMaskPlaneColor('SAT', afwDisplay.IGNORE)
        disp.setImageColormap('gray')
        disp.scale('linear', 'zscale')
        disp.mtv(isr_corr_exposure, title='visit = {}'.format(visit_int))
        cgUtils.overlayCcdBoxes(isr_corr_exposure.getDetector(), isTrimmed=True, display=disp)
    
    # Find all sources in the image
    tab = afwTable.SourceTable.make(schema)
    result = sourceDetectionTask.run(tab, isr_corr_exposure, sigma=2.1)
    
    # Find the correct sources
    zeroth_order_star_BBox= lsst.geom.Box2I.makeCenteredBox(zeroth_order_estimate, lsst.geom.Extent2I(200,200)) 
    # wavelength solution is bad for the fiberSpectrograph, but close enough for this to work
    # can use the monochromator wavelength which is better, but that's not the correct way in the long run

    dispersion = (1/0.6358) # pixels/nm
    spectral_position_angle=0.0107 # radians clockwise from top

    # Find 0th and +/- 1 order peaks 
    sources = result.sources
    center_source, peak1, peak2 = findNarrowbandRonchiPeaks(sources, zeroth_order_star_BBox, wavelength, dispersion, spectral_position_angle)
    
    # Fit peaks
    # zeroth order
    # variables names are weird here because I can't think of a clever way to have -1 and +1 as variable names
    
    bbox0 = lsst.geom.Box2I.makeCenteredBox(center_source.getFootprint().getCentroid(), lsst.geom.Extent2I(100,100)) 
    peak0_subim = isr_corr_exposure.subset(bbox0)
    p0, x0 , y0 = fit2DGaussian(peak0_subim, plot=True)
    p0_x_CofM, p0_y_CofM = calc_CofM(peak0_subim) # 2167,3372

    # Calculate EE and CofM
    p0_EE_rad50_pix, p0_EE_rad67_pix, p0_EE_rad80_pix = calc_encircled_energy(peak0_subim, plot=False) 
    fit_data.append(p0_EE_rad80_pix)

#     # offset to test
#     bbox1 = lsst.geom.Box2I.makeCenteredBox(peak1.getFootprint().getCentroid(), lsst.geom.Extent2I(50,50)) 
#     peak1_subim = isr_corr_exposure.subset(bbox1)
#     p1, x1 , y1 = fit2DGaussian(peak1_subim, plot=True)
#     p1_x_CofM, p1_y_CofM = calc_CofM(peak1_subim) # 2167,3372

#     # Calculate EE and CofM
#     p1_EE_rad50_pix, p1_EE_rad67_pix, p1_EE_rad80_pix = calc_encircled_energy(peak1_subim, plot=False) 
    
#     fit_data.append(p1_EE_rad80_pix)
#     bbox2 = lsst.geom.Box2I.makeCenteredBox(peak2.getFootprint().getCentroid(), lsst.geom.Extent2I(50,50)) 
#     peak2_subim = isr_corr_exposure.subset(bbox2)
#     p2, x2 , y2 = fit2DGaussian(peak2_subim, plot=False)
#     p2_x_CofM, p2_y_CofM = calc_CofM(peak2_subim) # 2167,3372

#     # Calculate EE and CofM
#     p0_EE_rad50_pix, p0_EE_rad67_pix, p0_EE_rad80_pix = calc_encircled_energy(peak0_subim, plot=False) 
#     p1_EE_rad50_pix, p1_EE_rad67_pix, p1_EE_rad80_pix = calc_encircled_energy(peak1_subim, plot=False) 
#     p2_EE_rad50_pix, p2_EE_rad67_pix, p2_EE_rad80_pix = calc_encircled_energy(peak2_subim, plot=False) 
